# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [2]:
import requests
import time
import re

from random import randint

In [3]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [4]:
import functools


def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter() - start
        print(f"function \"{func.__name__}\" took {round(end, 2)} seconds")
        return res
    
    return wrapper


@benchmark
def download(url=BOOK_PATH):
    """Функция загрузки исходного текста книги"""
    response = requests.get(url)
    print("status code is:", response.status_code)
    
download()

status code is: 200
function "download" took 1.89 seconds


## Задание 2

In [5]:
def logging(func):
    """
    Декоратор, который выводит параметры, с которыми была вызвана функция
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"function \"{func.__name__}\" called with args: \"{args}\" and kwargs: \"{kwargs}\"")
        res = func(*args, **kwargs)
        
        return res

    return wrapper

## Задание 3

In [6]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    def wrapper(*args, **kwargs):
        wrapper.cnt += 1
        print(f"function \"{func.__name__}\" was called {wrapper.cnt} times")
        res = func(*args, **kwargs)
        return res
    
    wrapper.cnt = 0

    return wrapper

## Задание 4

In [7]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}

  def fmemo(*args):
      cached_res = cache.get(args, None)
      if cached_res:
          return cached_res
      
      res = func(*args)
      cache[args] = res
      return res

  fmemo.cache = cache
  return fmemo

## Тестирование

In [8]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

function "word_count" was called 1 times
function "word_count" called with args: "('whole',)" and kwargs: "{}"
function "word_count" took 1.9 seconds
Cлово whole встречается 176 раз


In [9]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

### Комментарий
Раз уж функция уже определена, то продемонстрирую альтернативный вызов декоратора. Хотя предпочтительней было би при определении функции задать декорацию:
```python
@benchmark
def fib(n):
    ...
``` 

In [10]:
# Для проверки на одних и тех же данных
seed = 35

# измеряем время выполнения
benchmark(fib)(seed)

function "fib" took 2.4 seconds


9227465

In [11]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [12]:
# измеряем время выполнения
benchmark(fib)(100)

function "fmemo" took 0.0 seconds


354224848179261915075